# Adding a custom model to AutoGluon


This tutorial describes how to add a custom model to AutoGluon that can be trained, hyperparameter-tuned, and ensembled alongside the default models ([default model documentation](../../../api/autogluon.tabular.models.rst)).

In this example, we create a custom Random Forest model for use in AutoGluon. All models in AutoGluon inherit from the AbstractModel class ([AbstractModel source code](https://auto.gluon.ai/stable/_modules/autogluon/core/models/abstract/abstract_model.html)), and must follow its API to work alongside other models.

Note that while this tutorial provides a basic model implementation, this does not cover many aspects that are used in most implemented models.

To best understand how to implement more advanced functionality, refer to the [source code](../../../api/autogluon.tabular.models.rst) of the following models:

| Functionality | Reference Implementation |
| ------------- | ------------------------ |
| Respecting time limit / early stopping logic | [LGBModel](https://auto.gluon.ai/stable/_modules/autogluon/tabular/models/lgb/lgb_model.html) and [RFModel](https://auto.gluon.ai/stable/_modules/autogluon/tabular/models/rf/rf_model.html)
| Respecting memory usage limit | LGBModel and RFModel
| Sample weight support | LGBModel
| Validation data and eval_metric usage | LGBModel
| GPU training support | LGBModel
| Save / load logic of non-serializable models | [NNFastAiTabularModel](https://auto.gluon.ai/stable/_modules/autogluon/tabular/models/fastainn/tabular_nn_fastai.html)
| Advanced problem type support (Softclass, Quantile) | RFModel
| Text feature type support | [TextPredictorModel](https://auto.gluon.ai/stable/_modules/autogluon/tabular/models/text_prediction/text_prediction_v1_model.html)
| Image feature type support | [ImagePredictorModel](https://auto.gluon.ai/stable/_modules/autogluon/tabular/models/image_prediction/image_predictor.html)
| Lazy import of package dependencies | LGBModel
| Custom HPO logic | LGBModel

## Implementing a custom model

Here we define the custom model we will be working with for the rest of the tutorial.

The most important methods that must be implemented are `_fit` and `_preprocess`.


In [1]:
!pip install autogluon.tabular[all]


INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.0/71.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.0/821.0 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.3/201.3 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.7/155.7 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.3/487.3 kB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.0/103.0 k

In [2]:
import numpy as np
import pandas as pd

from autogluon.core.models import AbstractModel
from autogluon.features.generators import LabelEncoderFeatureGenerator

class CustomRandomForestModel(AbstractModel):
    def __init__(self, **kwargs):
        # Simply pass along kwargs to parent, and init our internal `_feature_generator` variable to None
        super().__init__(**kwargs)
        self._feature_generator = None

    # The `_preprocess` method takes the input data and transforms it to the internal representation usable by the model.
    # `_preprocess` is called by `preprocess` and is used during model fit and model inference.
    def _preprocess(self, X: pd.DataFrame, is_train=False, **kwargs) -> np.ndarray:
        print(f'Entering the `_preprocess` method: {len(X)} rows of data (is_train={is_train})')
        X = super()._preprocess(X, **kwargs)

        if is_train:
            # X will be the training data.
            self._feature_generator = LabelEncoderFeatureGenerator(verbosity=0)
            self._feature_generator.fit(X=X)
        if self._feature_generator.features_in:
            # This converts categorical features to numeric via stateful label encoding.
            X = X.copy()
            X[self._feature_generator.features_in] = self._feature_generator.transform(X=X)
        # Add a fillna call to handle missing values.
        # Some algorithms will be able to handle NaN values internally (LightGBM).
        # In those cases, you can simply pass the NaN values into the inner model.
        # Finally, convert to numpy for optimized memory usage and because sklearn RF works with raw numpy input.
        return X.fillna(0).to_numpy(dtype=np.float32)

    # The `_fit` method takes the input training data (and optionally the validation data) and trains the model.
    def _fit(self,
             X: pd.DataFrame,  # training data
             y: pd.Series,  # training labels
             # X_val=None,  # val data (unused in RF model)
             # y_val=None,  # val labels (unused in RF model)
             # time_limit=None,  # time limit in seconds (ignored in tutorial)
             **kwargs):  # kwargs includes many other potential inputs, refer to AbstractModel documentation for details
        print('Entering the `_fit` method')

        # First we import the required dependencies for the model. Note that we do not import them outside of the method.
        # This enables AutoGluon to be highly extensible and modular.
        # For an example of best practices when importing model dependencies, refer to LGBModel.
        from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

        # Valid self.problem_type values include ['binary', 'multiclass', 'regression', 'quantile', 'softclass']
        if self.problem_type in ['regression', 'softclass']:
            model_cls = RandomForestRegressor
        else:
            model_cls = RandomForestClassifier

        # Make sure to call preprocess on X near the start of `_fit`.
        # This is necessary because the data is converted via preprocess during predict, and needs to be in the same format as during fit.
        X = self.preprocess(X, is_train=True)
        # This fetches the user-specified (and default) hyperparameters for the model.
        params = self._get_model_params()
        print(f'Hyperparameters: {params}')
        # self.model should be set to the trained inner model, so that internally during predict we can call `self.model.predict(...)`
        self.model = model_cls(**params)
        self.model.fit(X, y)
        print('Exiting the `_fit` method')

    # The `_set_default_params` method defines the default hyperparameters of the model.
    # User-specified parameters will override these values on a key-by-key basis.
    def _set_default_params(self):
        default_params = {
            'n_estimators': 300,
            'n_jobs': -1,
            'random_state': 0,
        }
        for param, val in default_params.items():
            self._set_default_param_value(param, val)

    # The `_get_default_auxiliary_params` method defines various model-agnostic parameters such as maximum memory usage and valid input column dtypes.
    # For most users who build custom models, they will only need to specify the valid/invalid dtypes to the model here.
    def _get_default_auxiliary_params(self) -> dict:
        default_auxiliary_params = super()._get_default_auxiliary_params()
        extra_auxiliary_params = dict(
            # the total set of raw dtypes are: ['int', 'float', 'category', 'object', 'datetime']
            # object feature dtypes include raw text and image paths, which should only be handled by specialized models
            # datetime raw dtypes are generally converted to int in upstream pre-processing,
            # so models generally shouldn't need to explicitly support datetime dtypes.
            valid_raw_types=['int', 'float', 'category'],
            # Other options include `valid_special_types`, `ignored_type_group_raw`, and `ignored_type_group_special`.
            # Refer to AbstractModel for more details on available options.
        )
        default_auxiliary_params.update(extra_auxiliary_params)
        return default_auxiliary_params


## Loading the data

Next we will load the data. For this tutorial we will use the adult income dataset because it has a mix of integer, float, and categorical features.

In [3]:
from autogluon.tabular import TabularDataset

train_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/train.csv')  # can be local CSV file as well, returns Pandas DataFrame
test_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv')  # another Pandas DataFrame
label = 'class'  # specifies which column do we want to predict
train_data = train_data.sample(n=1000, random_state=0)  # subsample for faster demo

train_data.head(5)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
6118,51,Private,39264,Some-college,10,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,>50K
23204,58,Private,51662,10th,6,Married-civ-spouse,Other-service,Wife,White,Female,0,0,8,United-States,<=50K
29590,40,Private,326310,Some-college,10,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,44,United-States,<=50K
18116,37,Private,222450,HS-grad,9,Never-married,Sales,Not-in-family,White,Male,0,2339,40,El-Salvador,<=50K
33964,62,Private,109190,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,15024,0,40,United-States,>50K


## Training a custom model without TabularPredictor

Below we will demonstrate how to train the model outside [TabularPredictor](../../../api/autogluon.tabular.TabularPredictor.rst). This is useful for debugging and minimizing the amount of code you need to understand while implementing the model.

This process is similar to what happens internally when calling fit on `TabularPredictor`, but is simplified and minimal.

If the data was already cleaned (all numeric), then we could call fit directly with the data, but the adult dataset is not.

### Clean labels

The first step to making the input data as valid input to the model is to clean the labels.

Currently, they are strings, but we need to convert them to numeric values (0 and 1) for binary classification.

Luckily, AutoGluon already implements logic to both detect that this is binary classification (via `infer_problem_type`), and a converter to map the labels to 0 and 1 (`LabelCleaner`):

In [4]:
# Separate features and labels
X = train_data.drop(columns=[label])
y = train_data[label]
X_test = test_data.drop(columns=[label])
y_test = test_data[label]

from autogluon.core.data import LabelCleaner
from autogluon.core.utils import infer_problem_type
# Construct a LabelCleaner to neatly convert labels to float/integers during model training/inference, can also use to inverse_transform back to original.
problem_type = infer_problem_type(y=y)  # Infer problem type (or else specify directly)
label_cleaner = LabelCleaner.construct(problem_type=problem_type, y=y)
y_clean = label_cleaner.transform(y)

print(f'Labels cleaned: {label_cleaner.inv_map}')
print(f'inferred problem type as: {problem_type}')
print('Cleaned label values:')
y_clean.head(5)

Labels cleaned: {' <=50K': 0, ' >50K': 1}
inferred problem type as: binary
Cleaned label values:


,class
6118,1
23204,0
29590,0
18116,0
33964,1


### Clean features

Next, we need to clean the features. Currently, features like 'workclass' are object dtypes (strings), but we actually want to use them as categorical features. Most models won't accept string inputs, so we need to convert the strings to numbers.

AutoGluon contains an entire module dedicated to cleaning, transforming, and generating features called [autogluon.features](../../../api/autogluon.features.rst). Here we will use the same feature generator used internally by `TabularPredictor` to convert the object dtypes to categorical and minimize memory usage.

In [5]:
from autogluon.common.utils.log_utils import set_logger_verbosity
from autogluon.features.generators import AutoMLPipelineFeatureGenerator
set_logger_verbosity(2)  # Set logger so more detailed logging is shown for tutorial

feature_generator = AutoMLPipelineFeatureGenerator()
X_clean = feature_generator.fit_transform(X)

X_clean.head(5)

Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    11769.74 MB
	Train Data (Original)  Memory Usage: 0.51 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
			Note: Converting 1 features to boolean dtype as they only contain 2 unique values.
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
		Fitting CategoryFeatureGenerator...
			Fitting CategoryMemoryMinimizeFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('int', [])    : 6 | ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', ...]
		('object', []) : 8 | ['workclass', 'education', 'marital-sta

,age,fnlwgt,education-num,sex,capital-gain,capital-loss,hours-per-week,workclass,education,marital-status,occupation,relationship,race,native-country
6118,51,39264,10,0,0,0,40,3,14,1,4,5,4,24
23204,58,51662,6,0,0,0,8,3,0,1,8,5,4,24
29590,40,326310,10,1,0,0,44,3,14,1,3,0,4,24
18116,37,222450,9,1,0,2339,40,3,11,3,12,1,4,6
33964,62,109190,13,1,15024,0,40,3,9,1,4,0,4,24


[AutoMLPipelineFeatureGenerator](../../../api/autogluon.features.rst#AutoMLPipelineFeatureGenerator) does not fill missing values for numeric features nor does it rescale the values of numeric features or one-hot encode categoricals. If a model requires these operations, you'll need to add these operations into your `_preprocess` method, and may find some FeatureGenerator classes useful for this.

### Fit model

We are now ready to fit the model with the cleaned features and labels.

In [6]:
custom_model = CustomRandomForestModel()
# We could also specify hyperparameters to override defaults
# custom_model = CustomRandomForestModel(hyperparameters={'max_depth': 10})
custom_model.fit(X=X_clean, y=y_clean)  # Fit custom model

# To save to disk and load the model, do the following:
# load_path = custom_model.path
# custom_model.save()
# del custom_model
# custom_model = CustomRandomForestModel.load(path=load_path)

No path specified. Models will be saved in: "AutogluonModels/ag-20250909_212223/CustomRandomForestModel"
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Model CustomRandomForestModel's eval_metric inferred to be 'accuracy' because problem_type='binary' and eval_metric was not specified during init.


Entering the `_fit` method
Entering the `_preprocess` method: 1000 rows of data (is_train=True)
Hyperparameters: {'n_estimators': 300, 'n_jobs': -1, 'random_state': 0}
Exiting the `_fit` method


### Predict with trained model

Now that the model is fit, we can make predictions on new data. Remember that we need to perform the same data and label transformations to the new data as we did to the training data.

In [7]:
# Prepare test data
X_test_clean = feature_generator.transform(X_test)
y_test_clean = label_cleaner.transform(y_test)

X_test.head(5)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,31,Private,169085,11th,7,Married-civ-spouse,Sales,Wife,White,Female,0,0,20,United-States
1,17,Self-emp-not-inc,226203,12th,8,Never-married,Sales,Own-child,White,Male,0,0,45,United-States
2,47,Private,54260,Assoc-voc,11,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,1887,60,United-States
3,21,Private,176262,Some-college,10,Never-married,Exec-managerial,Own-child,White,Female,0,0,30,United-States
4,17,Private,241185,12th,8,Never-married,Prof-specialty,Own-child,White,Male,0,0,20,United-States


Get raw predictions from the test data

In [8]:
y_pred = custom_model.predict(X_test_clean)
print(y_pred[:5])

Entering the `_preprocess` method: 9769 rows of data (is_train=False)
[0 0 1 0 0]


Note that these predictions are of the positive class (whichever class was inferred to 1). To get more interpretable results, do the following:

In [9]:
y_pred_orig = label_cleaner.inverse_transform(y_pred)
y_pred_orig.head(5)

,0
0,<=50K
1,<=50K
2,>50K
3,<=50K
4,<=50K


### Score with trained model

By default, the model has an eval_metric specific to the problem_type. For binary classification, it uses accuracy.

We can get the accuracy score of the model by doing the following:

In [10]:
score = custom_model.score(X_test_clean, y_test_clean)
print(f'Test score ({custom_model.eval_metric.name}) = {score}')

Entering the `_preprocess` method: 9769 rows of data (is_train=False)
Test score (accuracy) = 0.8424608455317842


## Training a bagged custom model without TabularPredictor

Some of the more advanced functionality in AutoGluon such as bagging can be done very easily to models once they inherit from AbstractModel.

You can even bag your custom model in a couple lines of code. This is a quick way to get quality improvements on nearly any model:

In [11]:
from autogluon.core.models import BaggedEnsembleModel
bagged_custom_model = BaggedEnsembleModel(CustomRandomForestModel())
# Parallel folding currently doesn't work with a class not defined in a separate module because of underlying pickle serialization issue
# You don't need this following line if you put your custom model in a separate file and import it.
bagged_custom_model.params['fold_fitting_strategy'] = 'sequential_local'
bagged_custom_model.fit(X=X_clean, y=y_clean, k_fold=10)  # Perform 10-fold bagging
bagged_score = bagged_custom_model.score(X_test_clean, y_test_clean)
print(f'Test score ({bagged_custom_model.eval_metric.name}) = {bagged_score} (bagged)')
print(f'Bagging increased model accuracy by {round(bagged_score - score, 4) * 100}%!')

No path specified. Models will be saved in: "AutogluonModels/ag-20250909_212235/CustomRandomForestModel"
No path specified. Models will be saved in: "AutogluonModels/ag-20250909_212235/BaggedEnsembleModel"
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Model CustomRandomForestModel's eval_metric inferred to be 'accuracy' because problem_type='binary' and eval_metric was not specified during init.
Model 's eval_metric inferred to be 'accuracy' because problem_type='binary' and eval_metric was not specified during init.
	Fitting 10 child models (S1F1 - S1F10) | Fitting with SequentialLocalFoldFittingStrategy (sequential: cpus=2, gpus=0)
Model S1F1's eval_metric inferred to be 'accuracy' because problem_type='binary' and eval_metric was not specified during init.


Entering the `_fit` method
Entering the `_preprocess` method: 900 rows of data (is_train=True)
Hyperparameters: {'n_estimators': 300, 'n_jobs': -1, 'random_state': 0}
Exiting the `_fit` method
Entering the `_preprocess` method: 100 rows of data (is_train=False)


Model S1F2's eval_metric inferred to be 'accuracy' because problem_type='binary' and eval_metric was not specified during init.


Entering the `_fit` method
Entering the `_preprocess` method: 900 rows of data (is_train=True)
Hyperparameters: {'n_estimators': 300, 'n_jobs': -1, 'random_state': 0}


Model S1F3's eval_metric inferred to be 'accuracy' because problem_type='binary' and eval_metric was not specified during init.


...

Exiting the `_fit` method
Entering the `_preprocess` method: 100 rows of data (is_train=False)
Entering the `_fit` method
Entering the `_preprocess` method: 900 rows of data (is_train=True)
Hyperparameters: {'n_estimators': 300, 'n_jobs': -1, 'random_state': 0}


Model S1F9's eval_metric inferred to be 'accuracy' because problem_type='binary' and eval_metric was not specified during init.


Exiting the `_fit` method
Entering the `_preprocess` method: 100 rows of data (is_train=False)
Entering the `_fit` method
Entering the `_preprocess` method: 900 rows of data (is_train=True)
Hyperparameters: {'n_estimators': 300, 'n_jobs': -1, 'random_state': 0}


Model S1F10's eval_metric inferred to be 'accuracy' because problem_type='binary' and eval_metric was not specified during init.


Exiting the `_fit` method
Entering the `_preprocess` method: 100 rows of data (is_train=False)
Entering the `_fit` method
Entering the `_preprocess` method: 900 rows of data (is_train=True)
Hyperparameters: {'n_estimators': 300, 'n_jobs': -1, 'random_state': 0}
Exiting the `_fit` method
Entering the `_preprocess` method: 100 rows of data (is_train=False)
Entering the `_preprocess` method: 9769 rows of data (is_train=False)
Entering the `_preprocess` method: 9769 rows of data (is_train=False)
Entering the `_preprocess` method: 9769 rows of data (is_train=False)
Entering the `_preprocess` method: 9769 rows of data (is_train=False)
Entering the `_preprocess` method: 9769 rows of data (is_train=False)
Entering the `_preprocess` method: 9769 rows of data (is_train=False)
Entering the `_preprocess` method: 9769 rows of data (is_train=False)
Entering the `_preprocess` method: 9769 rows of data (is_train=False)
Entering the `_preprocess` method: 9769 rows of data (is_train=False)
Entering the 

Note that the bagged model trained 10 CustomRandomForestModels on different splits of the training data. When making a prediction, the bagged model averages the predictions from these 10 models.

## Training a custom model with TabularPredictor

While not using [TabularPredictor](../../../api/autogluon.tabular.TabularPredictor.rst) allows us to simplify the amount of code we need to worry about while developing and debugging our model, eventually we want to leverage TabularPredictor to get the most out of our model.

The code to train the model from the raw data is very simple when using TabularPredictor. There is no need to specify a LabelCleaner, FeatureGenerator, or a validation set, all of that is handled internally.

Here we train 3 CustomRandomForestModel with different hyperparameters.

In [12]:
from autogluon.tabular import TabularPredictor

# custom_hyperparameters = {CustomRandomForestModel: {}}  # train 1 CustomRandomForestModel Model with default hyperparameters
custom_hyperparameters = {CustomRandomForestModel: [{}, {'max_depth': 10}, {'max_features': 0.9, 'max_depth': 20}]}  # Train 3 CustomRandomForestModel with different hyperparameters
predictor = TabularPredictor(label=label).fit(train_data, hyperparameters=custom_hyperparameters)

No path specified. Models will be saved in: "AutogluonModels/ag-20250909_212252"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Mar 30 16:01:29 UTC 2025
CPU Count:          2
Memory Avail:       11.01 GB / 12.67 GB (86.8%)
Disk Space Avail:   62.60 GB / 107.72 GB (58.1%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='extreme' : New in v1.4: Massively better than 'best' on datasets <30000 samples by using new models meta-learned on https://tabarena.ai: TabPFNv2, TabICL, Mitra, and TabM. Absolute best accuracy. Requires a GPU. Recommended 64 GB CPU memory and 32+ GB GPU me

Entering the `_fit` method
Entering the `_preprocess` method: 800 rows of data (is_train=True)
Hyperparameters: {'n_estimators': 300, 'n_jobs': -1, 'random_state': 0}


	0.835	 = Validation score   (accuracy)
	0.89s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: CustomRandomForestModel_2 ...
	Fitting with cpus=2, gpus=0, mem=0.0/11.0 GB


Exiting the `_fit` method
Entering the `_preprocess` method: 200 rows of data (is_train=False)
Entering the `_fit` method
Entering the `_preprocess` method: 800 rows of data (is_train=True)
Hyperparameters: {'n_estimators': 300, 'n_jobs': -1, 'random_state': 0, 'max_depth': 10}


	0.845	 = Validation score   (accuracy)
	0.8s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: CustomRandomForestModel_3 ...
	Fitting with cpus=2, gpus=0, mem=0.0/11.0 GB


Exiting the `_fit` method
Entering the `_preprocess` method: 200 rows of data (is_train=False)
Entering the `_fit` method
Entering the `_preprocess` method: 800 rows of data (is_train=True)
Hyperparameters: {'n_estimators': 300, 'n_jobs': -1, 'random_state': 0, 'max_features': 0.9, 'max_depth': 20}


	0.84	 = Validation score   (accuracy)
	1.09s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	Ensemble Weights: {'CustomRandomForestModel_2': 1.0}
	0.845	 = Validation score   (accuracy)
	0.01s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 3.43s ... Best model: WeightedEnsemble_L2 | Estimated inference throughput: 1773.3 rows/s (200 batch size)
Disabling decision threshold calibration for metric `accuracy` due to having fewer than 10000 rows of validation data for calibration, to avoid overfitting (200 rows).
	`accuracy` is generally not improved through threshold calibration. Force calibration via specifying `calibrate_decision_threshold=True`.


Exiting the `_fit` method
Entering the `_preprocess` method: 200 rows of data (is_train=False)


TabularPredictor saved. To load, use: predictor = TabularPredictor.load("/content/AutogluonModels/ag-20250909_212252")


### Predictor leaderboard

Here we show the stats of each of the models trained. Notice that a WeightedEnsemble model was also trained. This model tries to combine the predictions of the other models to get a better validation score via ensembling.

In [13]:
predictor.leaderboard(test_data)

Entering the `_preprocess` method: 9769 rows of data (is_train=False)
Entering the `_preprocess` method: 9769 rows of data (is_train=False)
Entering the `_preprocess` method: 9769 rows of data (is_train=False)


,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CustomRandomForestModel_2,0.846044,0.845,accuracy,0.232186,0.111089,0.804755,0.232186,0.111089,0.804755,1,True,2
1,WeightedEnsemble_L2,0.846044,0.845,accuracy,0.235822,0.112784,0.810915,0.003636,0.001695,0.006160,2,True,4
2,CustomRandomForestModel,0.840414,0.835,accuracy,0.245951,0.110313,0.892873,0.245951,0.110313,0.892873,1,True,1
3,CustomRandomForestModel_3,0.828846,0.840,accuracy,0.209996,0.101370,1.087932,0.209996,0.101370,1.087932,1,True,3


### Predict with fit predictor

Here we predict with the fit predictor. This will automatically use the best model (the one with highest score_val) to predict.

In [14]:
y_pred = predictor.predict(test_data)
# y_pred = predictor.predict(test_data, model='CustomRandomForestModel_3')  # If we want a specific model to predict
y_pred.head(5)

Entering the `_preprocess` method: 9769 rows of data (is_train=False)


,class
0,<=50K
1,<=50K
2,>50K
3,<=50K
4,<=50K


## Hyperparameter tuning a custom model with TabularPredictor

We can easily hyperparameter tune custom models by specifying a hyperparameter search space in-place of exact values.

Here we hyperparameter tune the custom model for 20 seconds:

In [15]:
from autogluon.common import space
custom_hyperparameters_hpo = {CustomRandomForestModel: {
    'max_depth': space.Int(lower=5, upper=30),
    'max_features': space.Real(lower=0.1, upper=1.0),
    'criterion': space.Categorical('gini', 'entropy'),
}}
# Hyperparameter tune CustomRandomForestModel for 20 seconds
predictor = TabularPredictor(label=label).fit(train_data,
                                              hyperparameters=custom_hyperparameters_hpo,
                                              hyperparameter_tune_kwargs='auto',  # enables HPO
                                              time_limit=20)

No path specified. Models will be saved in: "AutogluonModels/ag-20250909_212257"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Mar 30 16:01:29 UTC 2025
CPU Count:          2
Memory Avail:       11.03 GB / 12.67 GB (87.0%)
Disk Space Avail:   62.59 GB / 107.72 GB (58.1%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='extreme' : New in v1.4: Massively better than 'best' on datasets <30000 samples by using new models meta-learned on https://tabarena.ai: TabPFNv2, TabICL, Mitra, and TabM. Absolute best accuracy. Requires a GPU. Recommended 64 GB CPU memory and 32+ GB GPU me

Entering the `_fit` method
Entering the `_preprocess` method: 800 rows of data (is_train=True)
Hyperparameters: {'n_estimators': 300, 'n_jobs': -1, 'random_state': 0, 'max_depth': 5, 'max_features': 0.1, 'criterion': 'gini'}
Exiting the `_fit` method
Entering the `_preprocess` method: 200 rows of data (is_train=False)
Entering the `_fit` method
Entering the `_preprocess` method: 800 rows of data (is_train=True)
Hyperparameters: {'n_estimators': 300, 'n_jobs': -1, 'random_state': 0, 'max_depth': 20, 'max_features': 0.7436704297351775, 'criterion': 'gini'}
Exiting the `_fit` method
Entering the `_preprocess` method: 200 rows of data (is_train=False)
Entering the `_fit` method
Entering the `_preprocess` method: 800 rows of data (is_train=True)
Hyperparameters: {'n_estimators': 300, 'n_jobs': -1, 'random_state': 0, 'max_depth': 8, 'max_features': 0.8625265649057129, 'criterion': 'entropy'}
Exiting the `_fit` method
Entering the `_preprocess` method: 200 rows of data (is_train=False)
Enteri

	Stopping HPO to satisfy time limit...
Fitted model: CustomRandomForestModel/T1 ...
	0.805	 = Validation score   (accuracy)
	0.78s	 = Training   runtime
	0.11s	 = Validation runtime
Fitted model: CustomRandomForestModel/T2 ...
	0.835	 = Validation score   (accuracy)
	1.23s	 = Training   runtime
	0.09s	 = Validation runtime
Fitted model: CustomRandomForestModel/T3 ...
	0.825	 = Validation score   (accuracy)
	1.6s	 = Training   runtime
	0.08s	 = Validation runtime
Fitted model: CustomRandomForestModel/T4 ...
	0.855	 = Validation score   (accuracy)
	1.14s	 = Training   runtime
	0.11s	 = Validation runtime
Fitted model: CustomRandomForestModel/T5 ...
	0.835	 = Validation score   (accuracy)
	0.81s	 = Training   runtime
	0.1s	 = Validation runtime


Exiting the `_fit` method
Entering the `_preprocess` method: 200 rows of data (is_train=False)


Fitted model: CustomRandomForestModel/T6 ...
	0.83	 = Validation score   (accuracy)
	0.94s	 = Training   runtime
	0.1s	 = Validation runtime
Fitted model: CustomRandomForestModel/T7 ...
	0.845	 = Validation score   (accuracy)
	1.03s	 = Training   runtime
	0.11s	 = Validation runtime
Fitted model: CustomRandomForestModel/T8 ...
	0.845	 = Validation score   (accuracy)
	0.81s	 = Training   runtime
	0.11s	 = Validation runtime
Fitted model: CustomRandomForestModel/T9 ...
	0.835	 = Validation score   (accuracy)
	0.83s	 = Training   runtime
	0.1s	 = Validation runtime
Fitted model: CustomRandomForestModel/T10 ...
	0.845	 = Validation score   (accuracy)
	1.05s	 = Training   runtime
	0.11s	 = Validation runtime
Fitted model: CustomRandomForestModel/T11 ...
	0.85	 = Validation score   (accuracy)
	0.96s	 = Training   runtime
	0.11s	 = Validation runtime
Fitted model: CustomRandomForestModel/T12 ...
	0.835	 = Validation score   (accuracy)
	0.92s	 = Training   runtime
	0.11s	 = Validation runtime


Entering the `_preprocess` method: 200 rows of data (is_train=False)
Entering the `_preprocess` method: 200 rows of data (is_train=False)
Entering the `_preprocess` method: 200 rows of data (is_train=False)
Entering the `_preprocess` method: 200 rows of data (is_train=False)
Entering the `_preprocess` method: 200 rows of data (is_train=False)
Entering the `_preprocess` method: 200 rows of data (is_train=False)
Entering the `_preprocess` method: 200 rows of data (is_train=False)
Entering the `_preprocess` method: 200 rows of data (is_train=False)
Entering the `_preprocess` method: 200 rows of data (is_train=False)
Entering the `_preprocess` method: 200 rows of data (is_train=False)
Entering the `_preprocess` method: 200 rows of data (is_train=False)
Entering the `_preprocess` method: 200 rows of data (is_train=False)
Entering the `_preprocess` method: 200 rows of data (is_train=False)
Entering the `_preprocess` method: 200 rows of data (is_train=False)


Fitting model: WeightedEnsemble_L2 ... Training model for up to 19.82s of the 0.40s of remaining time.
	Ensemble Weights: {'CustomRandomForestModel/T4': 1.0}
	0.855	 = Validation score   (accuracy)
	0.0s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 19.64s ... Best model: WeightedEnsemble_L2 | Estimated inference throughput: 1765.0 rows/s (200 batch size)
Disabling decision threshold calibration for metric `accuracy` due to having fewer than 10000 rows of validation data for calibration, to avoid overfitting (200 rows).
	`accuracy` is generally not improved through threshold calibration. Force calibration via specifying `calibrate_decision_threshold=True`.


Entering the `_preprocess` method: 200 rows of data (is_train=False)


TabularPredictor saved. To load, use: predictor = TabularPredictor.load("/content/AutogluonModels/ag-20250909_212257")


### Predictor leaderboard (HPO)

The leaderboard for the HPO run will show models with suffix `'/Tx'` in their name. This indicates the HPO trial they were performed in.

In [16]:
leaderboard_hpo = predictor.leaderboard()
leaderboard_hpo

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CustomRandomForestModel/T4,0.855,accuracy,0.111484,1.137711,0.111484,1.137711,1,True,4
1,WeightedEnsemble_L2,0.855,accuracy,0.113314,1.142133,0.001830,0.004423,2,True,16
2,CustomRandomForestModel/T11,0.850,accuracy,0.111331,0.957884,0.111331,0.957884,1,True,11
3,CustomRandomForestModel/T15,0.845,accuracy,0.086010,1.251602,0.086010,1.251602,1,True,15
4,CustomRandomForestModel/T8,0.845,accuracy,0.110682,0.807252,0.110682,0.807252,1,True,8
5,CustomRandomForestModel/T7,0.845,accuracy,0.111073,1.032054,0.111073,1.032054,1,True,7
6,CustomRandomForestModel/T10,0.845,accuracy,0.111609,1.054311,0.111609,1.054311,1,True,10
7,CustomRandomForestModel/T13,0.840,accuracy,0.099037,0.854633,0.099037,0.854633,1,True,13
8,CustomRandomForestModel/T14,0.835,accuracy,0.069083,1.062546,0.069083,1.062546,1,True,14
9,CustomRandomForestModel/T2,0.835,accuracy,0.088150,1.229579,0.088150,1.229579,1,True,2


### Getting the hyperparameters of a trained model

Let's get the hyperparameters of the model with the highest validation score.

In [17]:
best_model_name = leaderboard_hpo[leaderboard_hpo['stack_level'] == 1]['model'].iloc[0]

predictor_info = predictor.info()
best_model_info = predictor_info['model_info'][best_model_name]

print(best_model_info)

print(f'Best Model Hyperparameters ({best_model_name}):')
print(best_model_info['hyperparameters'])

{'name': 'CustomRandomForestModel/T4', 'model_type': 'CustomRandomForestModel', 'problem_type': 'binary', 'eval_metric': 'accuracy', 'stopping_metric': 'accuracy', 'fit_time': 1.1377105712890625, 'num_classes': 2, 'quantile_levels': None, 'predict_time': 0.11148428916931152, 'val_score': 0.855, 'hyperparameters': {'n_estimators': 300, 'n_jobs': -1, 'random_state': 0, 'max_depth': 26, 'max_features': 0.4459435365634299, 'criterion': 'entropy'}, 'hyperparameters_user': {'max_depth': 26, 'max_features': 0.4459435365634299, 'criterion': 'entropy', 'n_estimators': 300, 'n_jobs': -1, 'random_state': 0}, 'hyperparameters_fit': {}, 'hyperparameters_nondefault': ['max_depth', 'max_features', 'criterion', 'n_estimators', 'n_jobs', 'random_state'], 'ag_args_fit': {'max_memory_usage_ratio': 1.0, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': ['int', 'float', 'category'], 'valid_special_types': None, 'ignored_type_group_special': None, 'ignored_type_gro

## Training a custom model alongside other models with TabularPredictor

Finally, we will train the custom model (with tuned hyperparameters) alongside the default AutoGluon models.

All this requires is getting the hyperparameter dictionary of the default models via `get_hyperparameter_config`, and adding CustomRandomForestModel as a key.

In [18]:
from autogluon.tabular.configs.hyperparameter_configs import get_hyperparameter_config

# Now we can add the custom model with tuned hyperparameters to be trained alongside the default models:
custom_hyperparameters = get_hyperparameter_config('default')

custom_hyperparameters[CustomRandomForestModel] = best_model_info['hyperparameters']

print(custom_hyperparameters)

{'NN_TORCH': {}, 'GBM': [{'extra_trees': True, 'ag_args': {'name_suffix': 'XT'}}, {}, {'learning_rate': 0.03, 'num_leaves': 128, 'feature_fraction': 0.9, 'min_data_in_leaf': 3, 'ag_args': {'name_suffix': 'Large', 'priority': 0, 'hyperparameter_tune_kwargs': None}}], 'CAT': {}, 'XGB': {}, 'FASTAI': {}, 'RF': [{'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'squared_error', 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression', 'quantile']}}], 'XT': [{'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'squared_error', 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression', 'quantile']}}], <class '__main__.CustomRandomForestModel'>:

In [19]:
predictor = TabularPredictor(label=label).fit(train_data, hyperparameters=custom_hyperparameters)  # Train the default models plus a single tuned CustomRandomForestModel
# predictor = TabularPredictor(label=label).fit(train_data, hyperparameters=custom_hyperparameters, presets='best_quality')  # We can even use the custom model in a multi-layer stack ensemble
predictor.leaderboard(test_data)

No path specified. Models will be saved in: "AutogluonModels/ag-20250909_212323"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Mar 30 16:01:29 UTC 2025
CPU Count:          2
Memory Avail:       10.98 GB / 12.67 GB (86.6%)
Disk Space Avail:   62.54 GB / 107.72 GB (58.1%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='extreme' : New in v1.4: Massively better than 'best' on datasets <30000 samples by using new models meta-learned on https://tabarena.ai: TabPFNv2, TabICL, Mitra, and TabM. Absolute best accuracy. Requires a GPU. Recommended 64 GB CPU memory and 32+ GB GPU me

Entering the `_fit` method
Entering the `_preprocess` method: 800 rows of data (is_train=True)
Hyperparameters: {'n_estimators': 300, 'n_jobs': -1, 'random_state': 0, 'max_depth': 26, 'max_features': 0.4459435365634299, 'criterion': 'entropy'}


	0.855	 = Validation score   (accuracy)
	0.93s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...


Exiting the `_fit` method
Entering the `_preprocess` method: 200 rows of data (is_train=False)


	Ensemble Weights: {'LightGBMXT': 0.267, 'CustomRandomForestModel': 0.267, 'CatBoost': 0.2, 'RandomForestGini': 0.133, 'ExtraTreesEntr': 0.133}
	0.885	 = Validation score   (accuracy)
	0.13s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 34.95s ... Best model: WeightedEnsemble_L2 | Estimated inference throughput: 567.6 rows/s (200 batch size)
Disabling decision threshold calibration for metric `accuracy` due to having fewer than 10000 rows of validation data for calibration, to avoid overfitting (200 rows).
	`accuracy` is generally not improved through threshold calibration. Force calibration via specifying `calibrate_decision_threshold=True`.
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("/content/AutogluonModels/ag-20250909_212323")


Entering the `_preprocess` method: 9769 rows of data (is_train=False)


,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost,0.852902,0.860,accuracy,0.025082,0.012592,5.130339,0.025082,0.012592,5.130339,1,True,5
1,WeightedEnsemble_L2,0.850957,0.885,accuracy,0.833626,0.352378,15.034129,0.006688,0.000978,0.125264,2,True,13
2,LightGBMXT,0.850752,0.850,accuracy,0.084844,0.006408,6.846957,0.084844,0.006408,6.846957,1,True,1
3,NeuralNetFastAI,0.846965,0.850,accuracy,0.235266,0.014005,3.072960,0.235266,0.014005,3.072960,1,True,8
4,XGBoost,0.846658,0.855,accuracy,0.065530,0.007893,0.608258,0.065530,0.007893,0.608258,1,True,9
5,LightGBM,0.841335,0.840,accuracy,0.054076,0.005535,0.535338,0.054076,0.005535,0.535338,1,True,2
6,RandomForestGini,0.840004,0.840,accuracy,0.233148,0.111717,0.922118,0.233148,0.111717,0.922118,1,True,3
7,RandomForestEntr,0.837240,0.835,accuracy,0.255677,0.111531,1.033650,0.255677,0.111531,1.033650,1,True,4
8,NeuralNetTorch,0.836728,0.855,accuracy,0.080107,0.015849,10.973053,0.080107,0.015849,10.973053,1,True,10
9,CustomRandomForestModel,0.834988,0.855,accuracy,0.204168,0.111831,0.927615,0.204168,0.111831,0.927615,1,True,12


## Wrapping up

That's all it takes to add a custom model to AutoGluon. If you create a custom model, consider [submitting a PR](https://github.com/autogluon/autogluon/pulls) so that we can add it officially to AutoGluon!

For more tutorials, refer to [Predicting Columns in a Table - Quick Start](../tabular-quick-start.ipynb) and [Predicting Columns in a Table - In Depth](../tabular-indepth.ipynb).

For a tutorial on advanced custom models, refer to [Adding a custom model to AutoGluon (Advanced)](tabular-custom-model-advanced.ipynb))